In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf

from utilities import load_data

In [2]:
learning_rate = 0.1

batch_size = 100
kappaNN = 0.001*2*0

numwords = 3
vocab_size = 250
n_embed = 50
n_hidden = 200
n_output = vocab_size

In [11]:
TEST = False
if TEST:
    # Investigate *gather* operation performs as expected
    [train_input, train_target, valid_input, valid_target, test_input, test_target, vocab] = load_data(batch_size)
    sess = tf.InteractiveSession()
    batch_size = 100
    batch_labels = train_target[:,:,0].astype(int)
    u = tf.gather(tf.eye(vocab_size), batch_labels, validate_indices=None, name=None)
    v = tf.reshape(u, [batch_size,-1])
    _u = u.eval()
    _v = v.eval()
    #
    for i in range(5):
        print(i, np.sum(_u[0,i,:]), _u[0,i,batch_labels[0,i]])
    for i in range(5):
        print(i, np.sum(_v[i,:]), _v[i,batch_labels[0,i]])
    #
    batch_data = train_input[:,:,0].astype(int)
    print(batch_data.shape)
    W2E = np.array([[int(str(i+1)+'000'+str(j+1)) for j in range(n_embed)] for i in range(vocab_size)])
    weightsW2E = tf.constant(W2E.reshape(vocab_size, n_embed))
    #weightsW2E = tf.truncated_normal([vocab_size, n_embed])
    _weightsW2E = weightsW2E.eval()



    embed = tf.gather(weightsW2E, batch_data, validate_indices=None, name=None)
    embedR = tf.reshape(tf.transpose(embed, perm=[1,0,2]), [-1,numwords*n_embed])
    _embed = embed.eval()
    _embedR = embedR.eval()
    print(_embed.shape)
    print(_embedR.shape)
    #print(_embed[0,11,:])
    #print(_embed[1,11,:])
    print(_embed[2,11,:])
    #print(batch_data[:,11])
    print((_embed[0,11,:] == _embedR[11,0*n_embed:1*n_embed]).all())
    print((_embed[1,11,:] == _embedR[11,1*n_embed:2*n_embed]).all())
    print((_embed[2,11,:] == _embedR[11,2*n_embed:3*n_embed]).all())



    for b in range(batch_size):
        for j in range(n_embed):
            assert( _embed[0,b,j] == _weightsW2E[batch_data[0,b],j] )
    for b in range(batch_size):
        for j in range(n_embed):
            assert( _embedR[b,j+0*n_embed] == _weightsW2E[batch_data[0,b],j] )
            assert( _embedR[b,j+1*n_embed] == _weightsW2E[batch_data[1,b],j] )
            assert( _embedR[b,j+2*n_embed] == _weightsW2E[batch_data[2,b],j] )

In [23]:
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.int32, shape=(numwords, batch_size))
    tf_train_labels = tf.placeholder(tf.int32, shape=(1, batch_size))
    #tf_valid_dataset = tf.constant(valid_dataset)
    #tf_test_dataset = tf.constant(test_dataset)
    
    init_scale = 0.1

    # Variables.
    weightsW2E = tf.Variable(init_scale*tf.truncated_normal([vocab_size, n_embed]))
    weightsE2H = tf.Variable(init_scale*tf.truncated_normal([n_embed*numwords, n_hidden]))
    biasesH = tf.Variable(tf.zeros([n_hidden]))
    weightsH2O = tf.Variable(init_scale*tf.truncated_normal([n_hidden, n_output]))
    biasesO = tf.Variable(tf.zeros([n_output]))

    # Training computation.
    train_embed = tf.reshape(
        tf.transpose(
            tf.gather(weightsW2E, tf_train_dataset, validate_indices=None, name=None),
            perm=[1, 0, 2]),
        [-1, numwords*n_embed])
    train_logitsH = tf.matmul(train_embed, weightsE2H) + biasesH
    train_H = tf.sigmoid(train_logitsH)
    train_logitsO = tf.matmul(train_H, weightsH2O) + biasesO
    SOFTMAX_DIM = -1 # WARNING : dim across softmax is applied
    train_predictionO = tf.nn.softmax(train_logitsO, dim=SOFTMAX_DIM) # Info : not used in cross-entropy function
    #print(train_logitsO)
    #print(train_predictionO)
    #print(tf.gather(tf.eye(vocab_size), tf_train_labels, validate_indices=None, name=None))
    train_target = tf.reshape(tf.gather(tf.eye(vocab_size), tf_train_labels, validate_indices=None, name=None), [batch_size,-1])
    #print(train_target)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(train_logitsO, train_target, dim=SOFTMAX_DIM))
    param1_l2 = tf.nn.l2_loss(weightsE2H)# + tf.nn.l2_loss(biasesH)
    param2_l2 = tf.nn.l2_loss(weightsH2O)# + tf.nn.l2_loss(biasesO)
    reg_loss = loss + kappaNN * (param1_l2 + param2_l2)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(reg_loss)

    '''
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_logitsO)
    #
    valid_logitsH = tf.matmul(tf_valid_dataset, weightsE2H) + biasesE2H
    valid_H = tf.nn.relu(valid_logitsH)
    valid_logitsO = tf.matmul(valid_H, weightsH2O) + biasesH2O
    valid_predictionO = tf.nn.softmax(valid_logitsO)
    #
    test_logitsH = tf.matmul(tf_test_dataset, weightsE2H) + biasesE2H
    test_H = tf.nn.relu(test_logitsH)
    test_logitsO = tf.matmul(test_H, weightsH2O) + biasesH2O
    test_predictionO = tf.nn.softmax(test_logitsO)'''


In [24]:
from utilities import load_data

[train_input, train_target, valid_input, valid_target, test_input, test_target, vocab] = load_data(batch_size)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for epoch in range(5): # small value to debug
        print(epoch)
        for step in range(train_input.shape[2]):
            # Generate a minibatch.
            batch_data = train_input[:,:,step].astype(int)
            batch_labels = train_target[:,:,step].astype(int)
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_predictionO], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                #print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
0
Minibatch loss at step 0: 5.598677
Minibatch accuracy: 0.0%
Minibatch loss at step 500: 4.607826
Minibatch accuracy: 0.0%
Minibatch loss at step 1000: 4.250089
Minibatch accuracy: 0.0%
Minibatch loss at step 1500: 4.516531
Minibatch accuracy: 0.0%
Minibatch loss at step 2000: 4.340186
Minibatch accuracy: 0.0%
Minibatch loss at step 2500: 4.171938
Minibatch accuracy: 0.0%
Minibatch loss at step 3000: 3.976564
Minibatch accuracy: 0.0%
Minibatch loss at step 3500: 3.837094
Minibatch accuracy: 0.0%
1
Minibatch loss at step 0: 3.915903
Minibatch accuracy: 9.0%
Minibatch loss at step 500: 3.904302
Minibatch accuracy: 0.0%
Minibatch loss at step 1000: 3.688459
Minibatch accuracy: 0.0%
Minibatch loss at step 1500: 3.980611
Minibatch accuracy: 0.0%
Minibatch loss at step 2000: 3.769028
Minibatch accuracy: 2.0%
Minibatch loss at step 2500: 3.721257
Minibatch accuracy: 0.0%
Minibatch loss at step 3000: 3.502981
Minibatch accuracy: 0.0%
Minibatch loss at step 3500: 3.368258
Minibatch